In [42]:
import os
import cv2
from glob import glob
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from albumentations import *


In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
drive_path = '/content/drive/MyDrive/PFA' 

In [45]:
def create_dir(path): 
    """Create a directory"""
    if not os.path.exists(path):
        os.mkdir(path)

In [46]:
def load_data(path, split=0.2):
    dataset1_path = path + '/dataset1'
    
    
    images = sorted(glob(f"{dataset1_path}/Images/*.BMP"))
    masks_te = sorted(glob(f"{dataset1_path}/GT_TE/*.bmp"))
    masks_icm = sorted(glob(f"{dataset1_path}/GT_ICM/*.bmp"))
    masks_zp = sorted(glob(f"{dataset1_path}/GT_ZP/*.bmp"))
    
    
    """Split data"""
    split_size = int(len(images) * split)
    print(len(masks_zp), len(masks_icm), len(masks_te), len(masks_b), len(images))
   
    train_i, test_i = train_test_split(images, test_size=split_size, random_state=42)
    train_te, test_te = train_test_split(masks_te, test_size=split_size, random_state=42)
    train_zp, test_zp = train_test_split(masks_zp, test_size=split_size, random_state=42)
    train_icm, test_icm = train_test_split(masks_icm, test_size=split_size, random_state=42)
   
    return (train_i, test_i), (train_zp, test_zp), (train_icm, test_icm), (train_te, test_te)

In [47]:
def read_this(image_file):
   
    image_src = cv2.imread(image_file)

    image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
    
    return image_src

In [48]:
def enhance_contrast(image_matrix, bins=256):
    """https://msameeruddin.hashnode.dev/image-equalization-contrast-enhancing-in-python"""
    image_flattened = image_matrix.flatten()
    image_hist = np.zeros(bins)

    # frequency count of each pixel
    for pix in image_matrix:
        image_hist[pix] += 1

    # cummulative sum
    cum_sum = np.cumsum(image_hist)
    norm = (cum_sum - cum_sum.min()) * 255
    # normalization of the pixel values
    n_ = cum_sum.max() - cum_sum.min()
    uniform_norm = norm / n_
    uniform_norm = uniform_norm.astype('int')

    # flat histogram
    image_eq = uniform_norm[image_flattened]
    # reshaping the flattened matrix to its original shape
    image_eq = np.reshape(a=image_eq, newshape=image_matrix.shape)

    return image_eq

In [49]:
def GT_blastocoel(maskA,maskB,maskC,mergedmasks):
    #finding mask of the blastocoel zone
    #te,zp,icm

    merge1=cv2.add(maskA,maskB)
    part_merged=cv2.add(merge1,maskC)
    GT_C=mergedmasks+part_merged
    GT_C[GT_C<255]=0

    return GT_C

In [50]:
def addmasks(maskA, maskB, maskC):
    # adding zp,icm,te to deduce full cell mask

    merge1 = cv2.add(maskA, maskB)
    merged = cv2.add(merge1, maskC)
    merged = 255 - merged
    '''https://learnopencv.com/filling-holes-in-an-image-using-opencv-python-c/'''

    th, im_th = cv2.threshold(merged, 220, 255, cv2.THRESH_BINARY_INV);
    im_floodfill = im_th.copy()
    h, w = im_th.shape[:2]
    mask = np.zeros((h + 2, w + 2), np.uint8)
    cv2.floodFill(im_floodfill, mask, (0, 0), 255);
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    result = im_th | im_floodfill_inv
    merged = result
    return merged

In [51]:
def generate_mask(mask1, mask2, mask3):
  mask1[mask1 == 255] = 32
  mask2[mask2 == 255] = 64
  mask3[mask3 == 255] = 128
  merge1 = cv2.add(mask1, mask2)
  merged = cv2.add(merge1, mask3)
  merged = 255 - merged
  return merged


In [52]:
def resize_enhance(i, zp, icm, te,f,l,b):
    W = 256
    H = 256

    i = cv2.resize(i, (W, H))
    zp = cv2.resize(zp, (W, H))
    icm = cv2.resize(icm, (W, H))
    te = cv2.resize(te, (W, H))
    f = cv2.resize(f, (W, H))
    l = cv2.resize(l, (W, H))
    b = cv2.resize(b, (W, H))
    i = enhance_contrast(i)
    i = i.astype('uint8')

    return i, zp, icm, te, f, l,b

In [53]:
import csv
from google.colab.patches import cv2_imshow

In [54]:
def augment_data(images, masks_zp, masks_icm, masks_te,masks_b, new_path, train):
    """Performing data augmentation"""
   
    print(len(images))
    for idx, (i, zp, icm, te,b) in tqdm(enumerate(zip(images, masks_zp, masks_icm, masks_te,masks_b)), total=len(images)):

        name = i.split("/")[-1].split(".")[0]
                
        i = read_this(i)
        zp = read_this(zp)
        icm = read_this(icm)
        te = read_this(te)
        b=read_this(b)
        label_mask = generate_mask(zp, icm, te)
        width, height = i.shape[0], i.shape[1]
        # MERGING MASKS = full cell mask
        mergedmask = addmasks(zp, icm, te)
        #b = GT_blastocoel(i,zp,icm,mergedmask)
        

        if train:

            aug = Compose([

                HorizontalFlip(p=1)

            ], additional_targets={'mask_zp': 'mask', 'mask_icm': 'mask', 'mask_te': 'mask','full':'mask', 'label':'mask', 'mask_blas':'mask'})

            augmented = aug(image=i, mask_zp=zp, mask_icm=icm, mask_te=te, full=mergedmask, label= label_mask, mask_blas= b)
            i1 = augmented['image']
            zp1 = augmented['mask_zp']
            icm1 = augmented['mask_icm']
            te1 = augmented['mask_te']
            f1 = augmented['full']
            l1 = augmented['label']
            b1 = augmented['mask_blas']
            aug = Compose([

                VerticalFlip(p=1)

            ], additional_targets={'mask_zp': 'mask', 'mask_icm': 'mask', 'mask_te': 'mask','full':'mask', 'label':'mask', 'mask_blas':'mask'})

            augmented = aug(image=i, mask_zp=zp, mask_icm=icm, mask_te=te, full=mergedmask, label= label_mask, mask_blas= b)
            i2 = augmented['image']
            zp2 = augmented['mask_zp']
            icm2 = augmented['mask_icm']
            te2 = augmented['mask_te']
            f2 = augmented['full']
            l2 = augmented['label']
            b2 = augmented['mask_blas']
            aug = Compose([

                Transpose(1)

            ], additional_targets={'mask_zp': 'mask', 'mask_icm': 'mask', 'mask_te': 'mask','full':'mask', 'label':'mask', 'mask_blas':'mask'})

            augmented = aug(image=i, mask_zp=zp, mask_icm=icm, mask_te=te, full=mergedmask, label= label_mask, mask_blas= b)
            i3 = augmented['image']
            zp3 = augmented['mask_zp']
            icm3 = augmented['mask_icm']
            te3 = augmented['mask_te']
            f3 = augmented['full']
            l3 = augmented['label']
            b3 = augmented['mask_blas']
            aug = Compose([

                HorizontalFlip(p=1)

            ], additional_targets={'mask_zp': 'mask', 'mask_icm': 'mask', 'mask_te': 'mask','full':'mask', 'label':'mask', 'mask_blas':'mask'})
                

            augmented = aug(image=i3, mask_zp=zp3, mask_icm=icm3, mask_te=te3, full=f3, label= l3, mask_blas = b3)
            i4 = augmented['image']
            zp4 = augmented['mask_zp']
            icm4 = augmented['mask_icm']
            te4 = augmented['mask_te']
            f4 = augmented['full']
            l4 = augmented['label']
            b4 = augmented['mask_blas']
                

            I = [i, i1, i2, i3,i4]
            ZP = [zp, zp1, zp2, zp3,zp4]
            ICM = [icm, icm1, icm2, icm3,icm4]
            TE = [te, te1, te2, te3,te4]
            F = [mergedmask, f1, f2, f3, f4]
            L = [label_mask, l1, l2, l3, l4]
            B = [b, b1, b2, b3, b4]

        else:
            I = [i]
            ZP = [zp]
            ICM = [icm]
            TE = [te]
            F = [mergedmask]
            L= [label_mask]
            B = [b]

        index = 0
        real_size = []
        for i_aug, zp_aug, icm_aug, te_aug, f_aug, l_aug, b_aug in zip(I, ZP, ICM, TE,F,L,B):
            
            i_w, i_h = i_aug.shape[0], i_aug.shape[1]
            
            i_res, zp_res, icm_res, te_res, f_res, l_res, b_res = resize_enhance(i_aug, zp_aug, icm_aug, te_aug,f_aug,l_aug, b_aug)
            
            
            if len(I) == 1:
                i_name = f"{name}.bmp"
                zp_name = f"{name} ZP_Mask.bmp"
                icm_name = f"{name} ICM_Mask.bmp"
                te_name = f"{name} TE_Mask.bmp"
                f_name = f"{name} full.bmp"
                roi_name = f"{name} ROI.bmp"
                l_name = f"{name} label.bmp"
                b_name = f"{name} blastocoel.bmp"
            else:
                i_name = f"{name}_{index}.bmp"
                zp_name = f"{name} ZP_Mask_{index}.bmp"
                icm_name = f"{name} ICM_Mask_{index}.bmp"
                te_name = f"{name} TE_Mask_{index}.bmp"
                f_name = f"{name} full_{index}.bmp"
                roi_name = f"{name} ROI_{index}.bmp"
                l_name = f"{name} label_{index}.bmp"
                b_name = f"{name} blastocoel_{index}.bmp"

            i_path = os.path.join(new_path, "images/", i_name)
            zp_path = os.path.join(new_path, "GT_ZP/", zp_name)
            icm_path = os.path.join(new_path, "GT_ICM/", icm_name)
            te_path = os.path.join(new_path, "GT_TE/", te_name)
            f_path = os.path.join(new_path, "GT_full/", f_name)
            roi_path = os.path.join(new_path, "ROI/", roi_name)
            l_path = os.path.join(new_path, "label/", l_name)
            b_path = os.path.join(new_path, "GT_blastocoel/", b_name)

            real_size.append([i_name, i_w, i_h])
            
            with open('/content/drive/MyDrive/PFA/new_data/real_size.csv', 'a+', newline='\n') as file:
              writer = csv.writer(file)
              writer.writerows(real_size)

            
            roi = f_res * i_res
            cv2.imwrite(i_path, i_res)
            cv2.imwrite(roi_path, roi)
            cv2.imwrite(zp_path, zp_res)
            cv2.imwrite(icm_path, icm_res)
            cv2.imwrite(te_path, te_res)
            cv2.imwrite(f_path, f_res)
            cv2.imwrite(l_path, l_res)
            cv2.imwrite(b_path, b_res)

            index += 1


In [55]:
#Load the dataset
(train_i, test_i), (train_zp, test_zp), (train_icm, test_icm), (train_te, test_te)= load_data(drive_path)

dataset1_path = path + '/new_data/test'
dataset1_path = path + '/new_data/train'
masks_zp = sorted(glob(f"{dataset1_path}/GT_blastocoel/*.bmp"))

test_path = drive_path+"/new_data/test/"
train_path= drive_path+"/new_data/train/"
create_dir(train_path)
create_dir(test_path)
create_dir(test_path+"images/")
create_dir(train_path+"images/")
create_dir(test_path+"GT_ZP/")
create_dir(train_path+"GT_ZP/")
create_dir(test_path+"GT_ICM/")
create_dir(train_path+"GT_ICM/")
create_dir(test_path+"GT_TE/")
create_dir(train_path+"GT_TE/")
create_dir(test_path+"GT_full/")
create_dir(train_path+"GT_full/")
create_dir(test_path+"ROI/")
create_dir(train_path+"ROI/")
create_dir(test_path+"label/")
create_dir(train_path+"label/")
create_dir(test_path+"GT_blastocoel/")
create_dir(train_path+"GT_blastocoel/")

augment_data(test_i, test_zp, test_icm, test_te, test_b, test_path, train=False)
augment_data(train_i, train_zp, train_icm, train_te, train_b, train_path, train=True)

249 249 249 0 249


ValueError: ignored